In [4]:
import requests
from bs4 import BeautifulSoup
import csv
import lxml

#reference to original code https://www.youtube.com/watch?v=m4hEAhHHykI

def get_page(url):
    response = requests.get(url)
    #print(response.ok)
    #print(response.status_code)
        
    if not response.ok:
        print('Server responded: ', response. status_code)
    else:
        soup = BeautifulSoup(response.text, 'lxml')
    return soup


#should return (data, True) or (data, False). the true or false correspond to 
#if the get_detail_data should be run again but at a different sold date
def get_detail_data(soup, date_sold_index):
    #title 
    #price
    #items sold
    resultTitle = ''
    resultCurrency = ''
    resultPrice = ''
    doAgain = 0
    
    #title
    try:
        title = soup.find('h1', id = 'itemTitle')
        #this will clear out the extra span tag in front of the title that almost always says "Details about"
        title.span.clear()
        resultTitle = title.get_text()
    except:
        resultTitle = ''
    
    #price
    try:
        price = soup.find('span', id = 'prcIsum').text.strip()
        resultCurrency, resultPrice = price.split(' ')
    except:
        resultCurrency = ''
        resultPrice = ''
    #shipping cost
    try:
        shipping = soup.find('span', id = 'fshippingCost')
        resultShipping = shipping.get_text().strip()
    except:
        resultShipping = "Free (or not listed)"
        #try:
            #shipping = soup.find('div', id = 'shippingSummary')
            #print('well we got this far')
            #resultShipping = shipping.get_text()
        #except:
            #resultShipping = 'fastandfree failed'
        
    try:
        condition = soup.find('div', id = 'vi-itm-cond')
        resultCondition = condition.get_text()
    except:
        resultCondition = ''
        
    #the following should check for the date sold, if it cannot find it
    #it should enter the page that shows the dates of each 
    #transaction and count the number of times it must duplicate the item
    #should the item come up with multiple sales, this should return '0'
    #if '0' is read by the earlier call to this method, it should then call
    #this method however many times are needed
    try:
        ship_date = soup.find('span', id = 'bb_tlft')
        resultShip_Date = ship_date.get_text().strip()
    except:
        #this should check to see if it can find a link in place of date sold
        try:
            print('got to the first date check')
            #look for the link
            dates_link = soup.find('a', class_ = 'vi-txt-underline')
            print('date check success', dates_link)
            #make the soup for that link
            inside = get_page(dates_link.get('href'))
            print('got to inside')
            #make a list of dates the item was sold
            result = []
            holder = inside.find_all('td', align='left')
            print(holder, '9')
            if len(holder) >= 3:
                for i in range(len(holder)):
                    if i%3 == 2:
                        result.append(holder[i].get_text())
            resultShip_Date = result[date_sold_index]
            if date_sold_index == len(result):
                doAgain = 0
                print('big yeet')
            else:
                doAgain = len(result)
        except:
            resultShip_Date = 'error'
    
        
        
    #print('Title: ',resultTitle)
    #print('Price: ', resultCurrency, resultPrice)
    #print('Shipping Cost: ', resultShipping)    
    #print('Condition: ', resultCondition)
    
    data = {
        'title': resultTitle,
        'currency': resultCurrency,
        'price' : resultPrice,
        'shipping' : resultShipping,
        'condition' : resultCondition,
        'date' : resultShip_Date
    }
    #print('got to the data part', data)
    return (data, doAgain)

def get_index_data(soup):
    try:
        links = soup.find_all('a', class_='vip')
    except:
        links = []
        
    urls = [item.get('href') for item in links]
    return urls
    #print('soup')
def write_csv(data, url):
    with open('output.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        row = [data['title'], data['price'], data['shipping'], data['condition'], data['date'], url]
        
        #undo this if/else statement to include the currency after the number if it is not US currency
        #if data['currency'] == 'US':
            #row = [data['title'], data['price'], data['shipping'], data['condition'], url]
        #else:
            #row = [data['title'], data['price'] + data['currency'], data['shipping'], data['condition'], url]
        
        writer.writerow(row)
        #print('we out here writing to the file')
def main():
    url = 'https://www.ebay.com/itm/Luminox-Mens-Watch-Set-Navy-Seal-3950-Series-Yellow-Black-Strap-3955-SET/232487876786?epid=92853865&hash=item36215b78b2:g:WX0AAOSw7A9dH2EG'
    url = 'https://www.ebay.com/itm/Jaragar-Automatic-Yellow-Face-Triangle-Lamborghini-Style-Racing-Sport-Mens-Watch/233454675737?_trkparms=ispr%3D1&hash=item365afba719:g:FREAAOSw8gdeAnQ-&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qWOLbVo0a%2FfpCTfxFDxJLeTl6NrwV6twQmw%2FNLNQrxQeJL0f9cMbYFOtbjExgVrriZiB2w36PI0qAKDlPwwQyKayMCG1PQr8CUMszQ1pVKZD2zaKEVLGS4M2MOoSlJn0NJV%2BxAoH0Dq3JrUMbYXFqM0WEYVGpTlQm4l1pyy9XACTqDIyzPLLLUwrpoebaIE%2FNWYckfv7lCMbtI%2F1oHQ%2F4hVaffmI45mjd%2F0o4QjTGgWfuFieTf%2FjmounRwllobCxmLNVfLzNx8UpZr7oScVldYvnwq37wconwf4%2F8uJq5G4Ez30osFq%2F1EjU2T37y7HYhLAj0s3Vy0jq%2BxQ%2BUxR0bEDZUIukmh8%2BuAyOBrIuqKik67ScZ9P1OZgbnUAIepHsTriAGXrQQ5YA5LZdK7qv4DQZFfI7JYX0tlPh6p9zVGDfu3v3zacT04hSHzuXHyDdQIhECEw7rXfPgOIwseHIF01mhKx0jGSTjGCbPgeX99TzlUmVluaRfZBKvHt8ykEkBKEAwb4LUfWAnlMMaPiGtGeTpHO5w5tx%2Fl4vuYm9oVr8P5oIMz%2FwDkjveE7GWmDO0T1l6JOmO3%2F%2BB%2FyzY%2FEoUcKOXk%2FUN9WGfsQYWa%2FQo7MPQ%2FVHERVYhSUzVRM7219fQ8X4EZ4ZE1DJKHYE6SG6ahHvkyQwaGMsmiTOim8VbHQBPY58EDPZ2pVlurfQScaQCWQaYz2PjIP4h0JaEMF%2FATuDRoJ2TXg4K4Tqt%2BGBF2J0w%3D%3D&checksum=23345467573772a1c73970c74357b122d10c76f49089&enc=AQAEAAACQBPxNw%2BVj6nta7CKEs3N0qWOLbVo0a%2FfpCTfxFDxJLeTl6NrwV6twQmw%2FNLNQrxQeJL0f9cMbYFOtbjExgVrriZiB2w36PI0qAKDlPwwQyKayMCG1PQr8CUMszQ1pVKZD2zaKEVLGS4M2MOoSlJn0NJV%2BxAoH0Dq3JrUMbYXFqM0WEYVGpTlQm4l1pyy9XACTqDIyzPLLLUwrpoebaIE%2FNWYckfv7lCMbtI%2F1oHQ%2F4hVaffmI45mjd%2F0o4QjTGgWfuFieTf%2FjmounRwllobCxmLNVfLzNx8UpZr7oScVldYvnwq37wconwf4%2F8uJq5G4Ez30osFq%2F1EjU2T37y7HYhLAj0s3Vy0jq%2BxQ%2BUxR0bEDZUIukmh8%2BuAyOBrIuqKik67ScZ9P1OZgbnUAIepHsTriAGXrQQ5YA5LZdK7qv4DQZFfI7JYX0tlPh6p9zVGDfu3v3zacT04hSHzuXHyDdQIhECEw7rXfPgOIwseHIF01mhKx0jGSTjGCbPgeX99TzlUmVluaRfZBKvHt8ykEkBKEAwb4LUfWAnlMMaPiGtGeTpHO5w5tx%2Fl4vuYm9oVr8P5oIMz%2FwDkjveE7GWmDO0T1l6JOmO3%2F%2BB%2FyzY%2FEoUcKOXk%2FUN9WGfsQYWa%2FQo7MPQ%2FVHERVYhSUzVRM7219fQ8X4EZ4ZE1DJKHYE6SG6ahHvkyQwaGMsmiTOim8VbHQBPY58EDPZ2pVlurfQScaQCWQaYz2PjIP4h0JaEMF%2FATuDRoJ2TXg4K4Tqt%2BGBF2J0w%3D%3D&checksum=23345467573772a1c73970c74357b122d10c76f49089'    
    url = 'https://www.ebay.com/itm/1997-2011-AUDI-Wheel-CENTER-CAP-for-A4-A6-S6-A8-TT-8D0601165K-Rim-Hub-Grey/202848096540?hash=item2f3ab0511c:g:5qYAAOSwAGxd6-dT'
    
    url = 'https://www.ebay.com/sch/mygiftstop/m.html?_nkw=&_armrs=1&_ipg=&_from=&_pgn=0'
    
    url = 'https://www.ebay.com/sch/m.html?_from=&LH_Complete=1&LH_Sold=1_pgn%3D1&_ssn=vtraudt&_pgn=2&_skc=50&rt=nc'
    
    url = 'https://www.ebay.com/sch/suncoastcyclesports/m.html?item=163683037322&hash=item261c44f08a&vxp=mtr&LH_Complete=1&LH_Sold=1&rt=nc&_trksid=p2046732.m1684'
    
    #url = 'https://www.ebay.com/sch/mygiftstop/m.html?_nkw&_armrs=1&_ipg&_from&LH_Complete=1&LH_Sold=1&rt=nc&_trksid=p2046732.m1684'
    products = get_index_data(get_page(url))
    
    for link in products:
        
        #data is the original check for the page and stuff. 0 means there are no repeats (yet)
        data = get_detail_data(get_page(link), 0)
        print('yodle')
        #should data have more than 1 (or 0) repeats, it should loop and print out as many dates sold as there are
        if data[1] != 0:
            for i in range(data[1]):
                data2 = get_detail_data(get_page(link), i)
                print(data2[1])
                write_csv(data2[0], link)
        else:
            
            write_csv(data[0], link)
            print(data[1])
        
if __name__ == '__main__':
    main()

yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
got to the first date check
date check success <a class="vi-txt-underline" href="https://offer.ebay.com/ws/eBayISAPI.dll?ViewBidsLogin&amp;item=362666647541&amp;rt=nc&amp;_trksid=p2047675.l2564">1 sold</a>
got to inside
[] 9
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
got to the first date check
date check success <a class="vi-txt-underline" href="https://offer.ebay.com/ws/eBayISAPI.dll?ViewBidsLogin&amp;item=362890448728&amp;rt=nc&amp;_trksid=p2047675.l2564">1 sold</a>
got to inside
[] 9
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
yodle
0
got to the first date check
date check success <a class="vi-txt-underline" href="https://offer.ebay.com/ws/eBayISAPI.dll?ViewBidsLogin&amp;item=164051261143&a

In [82]:
url = 'https://offer.ebay.com/ws/eBayISAPI.dll?ViewBidsLogin&item=163635076377&rt=nc&_trksid=p2047675.l2564'
url = 'https://offer.ebay.com/ws/eBayISAPI.dll?ViewBidsLogin&item=163913863905&rt=nc&_trksid=p2047675.l2564'
soup2 = BeautifulSoup(requests.get(url).text, 'lxml')
result = []
#print(len(result_list))
holder = soup2.find_all('td', align='left')
if len(holder) >= 3:
    for i in range(len(holder)):
        if i%3 == 2:
            result.append(holder[i].get_text())
    #result = soup2.find_all('td', align='left')[].get_text()
print(result)

    

['Jan-30-20 12:54:59 PST']


In [40]:
for i in range(6):
    if i%3 == 2:
        print(i)
    else:
        print(i, '-')

0 -
1 -
2
3 -
4 -
5
